In [1]:
import os
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#parameters to clean up data

visit_length = 150 # anything less than 200 frames gets dropped
m = 0.4 # mean value of Prob(proboscis)

frames_btw_visits = 600

In [3]:
def MergeVisits(df, frames):
    idx = df[df['frames_inbetween'] <= frames].index.values
    if len(idx)>1:
        conseq = idx[:-1] - idx[1:]
        for c, ind in zip(conseq, idx[:-1]):
            if c is -1:
                new_val = df.loc[idx+1,'MothOut'].values
                df.loc[idx- 1, 'MothOut'] = new_val 
            else:
                new_val = df.loc[idx,'MothOut'].values
                df.loc[idx- 1, 'MothOut'] = new_val 
        if c != -1:
            new_val = df.loc[idx[-1],'MothOut']
            df.loc[idx[-1]- 1, 'MothOut'] = new_val
    else:
        new_val = df.loc[idx,'MothOut'].values
        df.loc[idx- 1, 'MothOut'] = new_val

    df = df.drop(idx, axis = 0)
    df = df.reset_index(drop = True)
    return(df)

In [4]:
fileloc = r"../dataFolders/DLCtrackingResults/"
filelist = glob.glob(fileloc + '*Stats.csv')

In [5]:
for file in filelist:
    df = pd.read_csv(file)
    df = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])
    name = os.path.basename(file)[:-32]
    #drop trials based  on length and prob(proboscis)
    filtered_df = df[df['mean'] > m].copy()
    filt_df = filtered_df[filtered_df.FramesInView > visit_length]
    
    filt_df = filt_df.reset_index(drop = True)
    
    
    #merge trials
    frames_inbetween =  filt_df.MothIN.values[1:] - filt_df.MothOut.values[:-1]
    frames_inbetween = frames_inbetween.astype(np.float64)
    frames_inbetween = np.insert(frames_inbetween, 0, np.nan, axis = 0)
    filt_df['frames_inbetween'] = frames_inbetween
    
    final_df = MergeVisits(filt_df, frames_btw_visits)
    final_df = final_df.drop(columns = ['mean', 'median', 'max', 'min', 'frames_inbetween', 'FramesInView'])
    final_df.to_csv('../dataFolders/DLCtrackingResults/AlternateStep3/' + name + '_trueTrials.csv')